In [ ]:
from dotenv import load_dotenv
import sys
import os
from IPython.display import Image, display
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
load_dotenv(override=True)

In [ ]:


from agents_forge.core_agent.agent import generate_core_agent

agent = await generate_core_agent()

display(Image(agent.get_graph().draw_mermaid_png()))
config = {"configurable": {"thread_id": "10"}}


In [3]:

res = await agent.ainvoke(
    {
        "messages": 
            [HumanMessage(content="czesc, zbuduj agenta ktory bedzie robic research na temat AI na zadanych stronach www dla tech leada. chce konkretne wskazowki zeby sie cialge rozwijac. interesuje mnie glownie praktyczna wiedza i wskazowki przy implementacji. zaproponuj agenta. o nic nie dopytuj")]},
    config
    )

In [ ]:
print(res["messages"][-1])

In [7]:
res = await agent.ainvoke({"messages": [HumanMessage(content="""
generate
""")]}, config)

In [ ]:
type(agent.get_state(config).values["agent_config"])

In [ ]:
from pprint import pprint

# Get the state snapshot
state_snapshot = agent.get_state(config)

# Format and print the state in a more readable way
state_dict = {
    "messages": [
        {
            "type": type(msg).__name__,
            "content": msg.content
        } for msg in state_snapshot.values['messages']
    ],
    "planned_step": state_snapshot.values.get('planned_step', 'N/A'),
    "thread_id": state_snapshot.metadata['thread_id'],
    "step": state_snapshot.metadata['step']
}

# Safely add agent_blueprint if it exists (checking both spellings)
if 'agent_bleuprint' in state_snapshot.values:
    state_dict["agent_blueprint"] = state_snapshot.values['agent_bleuprint']
elif 'agent_blueprint' in state_snapshot.values:
    state_dict["agent_blueprint"] = state_snapshot.values['agent_blueprint']
else:
    state_dict["agent_blueprint"] = "Not available"

pprint(state_dict)

In [ ]:
import json
from pathlib import Path

# Pobierz stan agenta
state_snapshot = agent.get_state(config)

# Pobierz obiekt agent_config
agent_config = state_snapshot.values.get('agent_config')

# Ścieżka do pliku
output_file = Path("agent_config.json")

# Sposób dla klas Pydantic
try:
    # Dla nowszych wersji Pydantic (v2+)
    json_data = agent_config.model_dump()
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(json_data, f, indent=4)
    print(f"Zapisano konfigurację do {output_file}")
except AttributeError:
    try:
        # Dla starszych wersji Pydantic (v1)
        json_data = agent_config.dict()
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(json_data, f, indent=4)
        print(f"Zapisano konfigurację do {output_file}")
    except AttributeError:
        print("Obiekt agent_config nie jest klasą Pydantic. Spróbuj metody 2.")

In [ ]:
from agents_forge.agents_generation.generator import generate_agent_from_config
import json
# Wczytaj konfigurację agenta z pliku JSON

with open("agent_config.json", "r", encoding="utf-8") as f:
    agent_config_data = json.load(f)

# Zaimportuj klasę AgentConfig do walidacji
from agents_forge.agents_generation.config_schema import AgentConfig

# Przekształć dane JSON na obiekt AgentConfig
agent_config = AgentConfig.model_validate(agent_config_data)


agent_2 = await generate_agent_from_config(agent_config)
display(Image(agent_2.get_graph().draw_mermaid_png()))



In [ ]:
result = await agent_2.ainvoke({"messages": [HumanMessage(content="do research on best langgraph practices")]})



In [ ]:
result